In [1]:
import numpy as np
import pandas as pd
import itertools

In [2]:
data = '../Data/'

In [3]:
data = pd.read_csv(data + 'clean_countries.csv').drop(columns=['Index'])
data

,Year,P/I,NAME,E-MAIL,AFFILIATION TYPE,AFFILIATION NAME,COUNTRY,TYPE,COUNTRY.1,START DATE,LENGTH (# OF DAYS),100 LEVEL (INTRO),200 LEVEL (INTERMEDIATE),300 LEVEL (ADVANCED)
0,2013,I,Mary Ruckelshaus,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
1,2013,I,Anne Guerry,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
2,2013,I,Spencer Wood,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
3,2013,I,Robert Griffin,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
4,2013,I,Jess Silver,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
5,2013,I,Douglas Denu,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
6,2013,P,Ana Liberoff,ana.liberoff@gmail.com,Gov't,CONICET - Centro Nacional Patagónico - Puerto ...,AR,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
7,2013,P,María Cecilia Brand,cecibrand@hotmail.com,Gov't,CONICET - Esquel,AR,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
8,2013,P,Jorge Kuroda,chimekuroda@yahoo.com.ar,NGO,Centro de Ecología Aplicada de Nequén,AR,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
9,2013,P,Daniel Fernandez,dfernandez.ush@gmail.com,Gov't,CONICET - Centro Austral de Investigaciones Ci...,AR,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN


In [4]:
data['TYPE'].unique()

array(['Nodal', 'Partner', 'Intro', 'Public', 'Regional', 'Symposium',
       'Seminar'], dtype=object)

In [5]:
data_list = [data[data['Year'] == year].drop(columns=['Year']) for year in range(2013,2019)]

In [6]:
# data = './data/'

In [7]:
# xls = pd.ExcelFile(data + 'PARTICIPANT LISTS - 2013-present.xlsx')
# data_list = [pd.read_excel(xls, '2013'), pd.read_excel(xls, '2014'), pd.read_excel(xls, '2015'), pd.read_excel(xls, '2016'), pd.read_excel(xls, '2017'), pd.read_excel(xls, '2018')]

In [8]:
data_list[0].head()

,P/I,NAME,E-MAIL,AFFILIATION TYPE,AFFILIATION NAME,COUNTRY,TYPE,COUNTRY.1,START DATE,LENGTH (# OF DAYS),100 LEVEL (INTRO),200 LEVEL (INTERMEDIATE),300 LEVEL (ADVANCED)
0,I,Mary Ruckelshaus,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
1,I,Anne Guerry,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
2,I,Spencer Wood,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
3,I,Robert Griffin,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
4,I,Jess Silver,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN


In [9]:
# returns list of participants, instructors, and participant count by country for each year
def extract(df_list):
    participants = []
    instructors = []
    country_counts = []
    
    for df in df_list:
        temp = df[(df['P/I'] == 'P')]
        country_counts.append(temp.groupby(['COUNTRY']).size())
        participants.append(temp)
        instructors.append(df[(df['P/I'] == 'I')])
        
    return participants, instructors, country_counts

participants, instructors, country_counts = extract(data_list)

In [10]:
aggregated_data = pd.concat(country_counts, axis=1, sort=True).reset_index().fillna(value=0)
column_mapping = {'index': 'Country', 0: '2013', 1: '2014', 2: '2015', 3: '2016', 4: '2017', 5: '2018'}
aggregated_data = aggregated_data.rename(index=str, columns=column_mapping)
aggregated_data = pd.melt(aggregated_data, id_vars=['Country'], value_vars=["2013", "2014", "2015", "2016", "2017", "2018"])
aggregated_data = aggregated_data.sort_values(by=['Country', 'variable']).reset_index(drop=True)
aggregated_data = aggregated_data.rename(columns={'variable': 'Year', 'value':"Number of Participants"})
aggregated_data.head()


,Country,Year,Number of Participants
0,AE,2013,0.0
1,AE,2014,0.0
2,AE,2015,1.0
3,AE,2016,0.0
4,AE,2017,0.0


In [11]:
aggregated_data.to_csv('Aggregated Data.csv', sep=',', encoding='utf-8', index=False)

In [12]:
data_list[0].head()

,P/I,NAME,E-MAIL,AFFILIATION TYPE,AFFILIATION NAME,COUNTRY,TYPE,COUNTRY.1,START DATE,LENGTH (# OF DAYS),100 LEVEL (INTRO),200 LEVEL (INTERMEDIATE),300 LEVEL (ADVANCED)
0,I,Mary Ruckelshaus,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
1,I,Anne Guerry,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
2,I,Spencer Wood,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
3,I,Robert Griffin,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN
4,I,Jess Silver,NaN,Academic,Natural Capital Project,US,Nodal,AR,2013-09-12,2.0,1.0,1.0,NaN


In [13]:
names_only = [year['NAME'] for year in participants]
names_combined = pd.concat(names_only)
print("Total number of people trained over the years: " + str(len(names_combined)))

Total number of people trained over the years: 2791


In [14]:
names_only = [year['NAME'] for year in instructors]
names_combined = pd.concat(names_only)
print("Total number of trainees over the years: " + str(len(names_combined)))

Total number of trainees over the years: 472


In [15]:
countries_taught_in_only = [year['COUNTRY.1'] for year in instructors]
countries_taught_in_only_combined = pd.concat(countries_taught_in_only).unique()
print("Number of countries trained in over the years: " + str(len(countries_taught_in_only_combined)))
print()
print("List of countries trained in:\n" + ' '.join(countries_taught_in_only_combined))

Number of countries trained in over the years: 29

List of countries trained in:
AR KH CL GB MZ PE ID US CA PA MN NI ES CR KR IN BB BT KE CO IT CH EC ZA RO CN BR MM FI


In [16]:
print("Number of countries trainees come from over the years: " + str(len(aggregated_data['Country'].unique())))
print()
print("List of countries trained in:\n" + ' '.join(aggregated_data['Country'].unique()))

Number of countries trainees come from over the years: 89

List of countries trained in:
AE AR AT AU BB BD BE BG BJ BO BR BS BT BW BZ CA CH CI CL CM CN CO CR DE DK DO EC EG ES ET FR GB GE GH GR GT HU ID IL IN IT JM JO JP KE KG KH KR KZ LA LK LR LT MA MG MM MN MQ MX MY MZ NG NI NL NO NP NZ PA PE PH PK PR PT RO RU SC SE SG SI SR SZ TH TN TT TZ US VN ZA ZW


In [17]:
grouped_by_type = [year.groupby(['TYPE']).size() for year in participants]
training_type_per_year = pd.concat(grouped_by_type, axis=1, sort=True).reset_index().fillna(value=0)
column_mapping = {'index': 'Type', 0: '2013', 1: '2014', 2: '2015', 3: '2016', 4: '2017', 5: '2018'}
training_type_per_year = training_type_per_year.rename(index=str, columns=column_mapping)
training_type_per_year.head(10)

,Type,2013,2014,2015,2016,2017,2018
0,Intro,131.0,110.0,22.0,0.0,0.0,0.0
1,Nodal,71.0,32.0,64.0,0.0,0.0,8.0
2,Partner,272.0,25.0,195.0,71.0,392.0,369.0
3,Public,38.0,0.0,0.0,0.0,0.0,0.0
4,Regional,77.0,87.0,0.0,46.0,10.0,0.0
5,Seminar,0.0,0.0,14.0,0.0,0.0,0.0
6,Symposium,0.0,0.0,0.0,222.0,262.0,273.0


In [18]:
print([len(year['COUNTRY'].unique()) for year in participants])

[43, 33, 42, 41, 32, 41]


In [19]:
intro = pd.DataFrame([len(year['100 LEVEL\n(INTRO)'].dropna()) for year in participants]).transpose()
column_mapping = {0: '2013', 1: '2014', 2: '2015', 3: '2016', 4: '2017', 5: '2018'}
intro = intro.rename(columns=column_mapping, index={0: '100 Level (Intro)'})

intermediate = pd.DataFrame([len(year['200 LEVEL\n(INTERMEDIATE)'].dropna()) for year in participants]).transpose()
intermediate = intermediate.rename(columns=column_mapping, index={0: '200 Level (Intermediate)'})

advanced = pd.DataFrame([len(year['300 LEVEL\n(ADVANCED)'].dropna()) for year in participants]).transpose()
advanced = advanced.rename(columns=column_mapping, index={0: '300 Level (Advanced)'})

intro = intro.append(intermediate)
intro = intro.append(advanced)
intro.head()

,2013,2014,2015,2016,2017,2018
100 Level (Intro),587,240,291,339,664,650
200 Level (Intermediate),362,152,255,325,664,650
300 Level (Advanced),125,47,79,325,664,642


In [20]:
level_count_by_country = [year.groupby('COUNTRY').sum().drop('LENGTH\n(# OF DAYS)', 1) for year in participants]
level_count_by_country[0]['Year'] = 2013
level_count_by_country[1]['Year'] = 2014
level_count_by_country[2]['Year'] = 2015
level_count_by_country[3]['Year'] = 2016
level_count_by_country[4]['Year'] = 2017
level_count_by_country[5]['Year'] = 2018

level_count_by_country = [year.reset_index().groupby(['Year', 'COUNTRY']).sum() for year in level_count_by_country]
levels_grouped_by_year_and_country = pd.concat(level_count_by_country).reset_index()
levels_grouped_by_year_and_country.to_csv('Class Level Stats.csv', sep=',', encoding='utf-8', index=False)
levels_grouped_by_year_and_country.head()

,Year,COUNTRY,100 LEVEL (INTRO),200 LEVEL (INTERMEDIATE),300 LEVEL (ADVANCED)
0,2013,AR,28.0,27.5,0.0
1,2013,AU,12.0,3.0,0.0
2,2013,BE,2.0,2.0,1.0
3,2013,BG,2.0,0.5,0.0
4,2013,BR,14.5,11.0,4.0


---

In [108]:
level_stats = levels_grouped_by_year_and_country.melt(id_vars=['Year', 'COUNTRY'], var_name='Type', value_name='Count')
level_stats = level_stats.replace({'100 LEVEL\n(INTRO)':'Intro', 
                     '200 LEVEL\n(INTERMEDIATE)': 'Intermediate',
                     '300 LEVEL\n(ADVANCED)': 'Advanced'})
level_stats = level_stats.rename(columns={'COUNTRY': 'Country'})
level_stats

,Year,Country,Type,Count
0,2013,AR,Intro,28.0
1,2013,AU,Intro,12.0
2,2013,BE,Intro,2.0
3,2013,BG,Intro,2.0
4,2013,BR,Intro,14.5
5,2013,CA,Intro,19.5
6,2013,CH,Intro,7.0
7,2013,CL,Intro,24.0
8,2013,CM,Intro,2.0
9,2013,CN,Intro,6.0


In [109]:
years = range(2013, 2019)
list(years)

[2013, 2014, 2015, 2016, 2017, 2018]

In [110]:
countries = level_stats['Country'].unique()
countries

array(['AR', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CL', 'CM', 'CN', 'CO',
       'CR', 'DE', 'DK', 'DO', 'EC', 'ES', 'GB', 'GE', 'GT', 'HU', 'ID',
       'IT', 'JP', 'KE', 'KG', 'KH', 'LA', 'MN', 'MX', 'MY', 'MZ', 'NI',
       'NL', 'NZ', 'PA', 'PE', 'PH', 'PT', 'SE', 'SG', 'US', 'VN', 'BB',
       'BO', 'FR', 'GR', 'IL', 'IN', 'KR', 'LT', 'NO', 'NP', 'TT', 'AE',
       'AT', 'BD', 'BT', 'BW', 'EG', 'ET', 'GH', 'JO', 'KZ', 'LK', 'MA',
       'MM', 'PR', 'RO', 'SI', 'TH', 'TN', 'TZ', 'ZA', 'ZW', 'BJ', 'BS',
       'BZ', 'MQ', 'RU', 'SR', 'SZ', 'MG', 'NG', 'SC', 'CI', 'JM', 'LR',
       'PK'], dtype=object)

In [111]:
levels = level_stats['Type'].unique()
levels

array(['Intro', 'Intermediate', 'Advanced'], dtype=object)

In [112]:
empty = pd.DataFrame(list(itertools.product(years, countries, levels, [0])), columns=['Year', 'Country', 'Type', 'Count'])
display(empty.head(), empty.shape)

,Year,Country,Type,Count
0,2013,AR,Intro,0
1,2013,AR,Intermediate,0
2,2013,AR,Advanced,0
3,2013,AU,Intro,0
4,2013,AU,Intermediate,0


(1602, 4)

In [113]:
level_stats = level_stats.append(empty).groupby(['Year', 'Country', 'Type']).max()
display(level_stats.shape)
level_stats.head(16)

(1602, 1)

Count
Year Country Type               
2013 AE      Advanced        0.0
             Intermediate    0.0
             Intro           0.0
     AR      Advanced        0.0
             Intermediate   27.5
             Intro          28.0
     AT      Advanced        0.0
             Intermediate    0.0
             Intro           0.0
     AU      Advanced        0.0
             Intermediate    3.0
             Intro          12.0
     BB      Advanced        0.0
             Intermediate    0.0
             Intro           0.0
     BD      Advanced        0.0

In [114]:
### Trainee/days
data = pd.read_excel('../Data/PARTICIPANT LISTS - 2013-present.xlsx', sheet_name=None, header=1)
attendanceTD = pd.DataFrame()

# years 
for yr in data.keys():
    attendanceTD = attendanceTD.append({'Year': yr,
                                    
                                    # number of participants (P) for year yr
                                    'Total':data[yr][data[yr]['P/I']=='P']['LENGTH\n(# OF DAYS)'].sum(),
                                    
                                    # number of participants for year yr per type
                                    'Intro' : data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Intro')]['LENGTH\n(# OF DAYS)'].sum(),
                                    'Nodal' : data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Nodal')]['LENGTH\n(# OF DAYS)'].sum(),
                                    'Partner': data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Partner')]['LENGTH\n(# OF DAYS)'].sum(),
                                    'Public': data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Public')]['LENGTH\n(# OF DAYS)'].sum(),
                                    'Regional': data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Regional')]['LENGTH\n(# OF DAYS)'].sum(),
                                    'Seminar': data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Seminar')]['LENGTH\n(# OF DAYS)'].sum(),
                                    'Symposium': data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Symposium')]['LENGTH\n(# OF DAYS)'].sum(),
                                    'Webinar': data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Webinar')]['LENGTH\n(# OF DAYS)'].sum(),
                                    #'Other training': data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='nan')]['LENGTH\n(# OF DAYS)'].sum()
                                    
                                    # number of participants for year yr per level
                                    'Intro level' : data[yr][data[yr]['P/I']=='P']['100 LEVEL\n(INTRO)'].sum(),
                                    'Intermediate' : data[yr][data[yr]['P/I']=='P']['200 LEVEL\n(INTERMEDIATE)'].sum(),
                                    'Advanced' : data[yr][data[yr]['P/I']=='P']['300 LEVEL\n(ADVANCED)'].sum()
                                    },
                                     
                                   ignore_index=True)

attendanceTD['Other trainings'] = attendanceTD['Total'] - attendanceTD['Symposium']   
attendanceTD = attendanceTD.sort_values('Year')
attendanceTD = attendanceTD.set_index('Year')

# Get totals
# attendanceTD = attendanceTD.append(pd.Series(attendanceTD.sum(numeric_only=True), name='Total'), ignore_index=False)

attendanceTD

,Advanced,Intermediate,Intro,Intro level,Nodal,Partner,Public,Regional,Seminar,Symposium,Total,Webinar,Other trainings
Year,,,,,,,,,,,,,
2013,139.0,509.75,114.75,973.750,163.5,705.5,95.0,237.5,0.0,300.0,1622.500,6.250,1322.500
2014,58.5,368.00,109.50,507.810,54.0,179.0,0.0,280.5,0.0,360.0,1013.310,30.310,653.310
2015,93.5,637.00,13.00,849.497,204.0,750.0,0.0,0.0,70.0,624.0,1668.497,7.497,1044.497
2016,15.0,90.50,0.00,913.500,0.0,264.5,0.0,66.5,0.0,688.0,1019.000,0.000,331.000
2017,8.0,389.00,0.00,1727.000,0.0,942.0,0.0,126.0,0.0,1056.0,2124.000,0.000,1068.000
2018,0.0,340.00,0.00,1770.000,18.0,1221.0,0.0,48.0,0.0,823.0,2110.000,0.000,1287.000


In [115]:
attendanceTD = attendanceTD.loc[:, ['Intro level', 'Intermediate', 'Advanced']]
attendanceTD['Country'] = 'WORLD'
attendanceTD = attendanceTD.rename(columns={'Intro level':'Intro'}).reset_index()
world_stats = attendanceTD.melt(id_vars=['Year', 'Country'], var_name='Type', value_name='Count')
world_stats['Year'] = pd.to_numeric(world_stats['Year'])
world_stats

,Year,Country,Type,Count
0,2013,WORLD,Intro,973.750
1,2014,WORLD,Intro,507.810
2,2015,WORLD,Intro,849.497
3,2016,WORLD,Intro,913.500
4,2017,WORLD,Intro,1727.000
5,2018,WORLD,Intro,1770.000
6,2013,WORLD,Intermediate,509.750
7,2014,WORLD,Intermediate,368.000
8,2015,WORLD,Intermediate,637.000
9,2016,WORLD,Intermediate,90.500


In [116]:
level_stats = level_stats.reset_index().append(world_stats).groupby(['Year', 'Country', 'Type']).max()
level_stats

Count
Year Country Type                
2013 AE      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     AR      Advanced         0.0
             Intermediate    27.5
             Intro           28.0
     AT      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     AU      Advanced         0.0
             Intermediate     3.0
             Intro           12.0
     BB      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     BD      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     BE      Advanced         1.0
             Intermediate     2.0
             Intro            2.0
     BG      Advanced         0.0
             Intermediate     0.5
             Intro            2.0
     BJ      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     BO      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
...                           ...
2018 SZ      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     TH      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     TN      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     TT      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     TZ      Advanced         0.0
             Intermediate     0.0
             Intro            3.0
     US      Advanced         0.0
             Intermediate   167.0
             Intro          346.0
     VN      Advanced         0.0
             Intermediate     0.0
             Intro            0.0
     WORLD   Advanced         0.0
             Intermediate   340.0
             Intro         1770.0
     ZA      Advanced         0.0
             Intermediate     1.0
             Intro            2.0
     ZW      Advanced         0.0
             Intermediate     0.0
             Intro            0.0

[1620 rows x 1 columns]

Add totals for each country and the world:

In [117]:
totals = level_stats.groupby(['Country', 'Type']).sum().reset_index()
totals['Year'] = 'Total'
cols = totals.columns.tolist()
cols = cols[-1:] + cols[:-1]
totals = totals[cols]
totals

,Year,Country,Type,Count
0,Total,AE,Advanced,1.000
1,Total,AE,Intermediate,1.500
2,Total,AE,Intro,0.500
3,Total,AR,Advanced,0.000
4,Total,AR,Intermediate,28.500
5,Total,AR,Intro,30.000
6,Total,AT,Advanced,0.000
7,Total,AT,Intermediate,4.000
8,Total,AT,Intro,2.000
9,Total,AU,Advanced,0.000


In [118]:
level_stats = level_stats.reset_index().append(totals).groupby(['Year', 'Country', 'Type']).max()
level_stats = level_stats.round()
level_stats

Count
Year  Country Type                
2013  AE      Advanced         0.0
              Intermediate     0.0
              Intro            0.0
      AR      Advanced         0.0
              Intermediate    28.0
              Intro           28.0
      AT      Advanced         0.0
              Intermediate     0.0
              Intro            0.0
      AU      Advanced         0.0
              Intermediate     3.0
              Intro           12.0
      BB      Advanced         0.0
              Intermediate     0.0
              Intro            0.0
      BD      Advanced         0.0
              Intermediate     0.0
              Intro            0.0
      BE      Advanced         1.0
              Intermediate     2.0
              Intro            2.0
      BG      Advanced         0.0
              Intermediate     0.0
              Intro            2.0
      BJ      Advanced         0.0
              Intermediate     0.0
              Intro            0.0
      BO      Advanced         0.0
              Intermediate     0.0
              Intro            0.0
...                            ...
Total SZ      Advanced         0.0
              Intermediate     0.0
              Intro            1.0
      TH      Advanced         0.0
              Intermediate    18.0
              Intro            8.0
      TN      Advanced         4.0
              Intermediate     8.0
              Intro            3.0
      TT      Advanced         0.0
              Intermediate     0.0
              Intro            0.0
      TZ      Advanced         0.0
              Intermediate     2.0
              Intro            4.0
      US      Advanced        44.0
              Intermediate   502.0
              Intro         1538.0
      VN      Advanced         3.0
              Intermediate    14.0
              Intro           14.0
      WORLD   Advanced       314.0
              Intermediate  2334.0
              Intro         6742.0
      ZA      Advanced         0.0
              Intermediate    40.0
              Intro           27.0
      ZW      Advanced         0.0
              Intermediate     2.0
              Intro            1.0

[1890 rows x 1 columns]

Sanity check:

In [119]:
len(levels) * (len(years) + 1) * (len(countries) + 1) == len(level_stats)

True

Save:

In [121]:
level_stats.to_csv('../Data/Level_Stats.csv')

---

In [48]:
type_count_by_country = [year.groupby(['COUNTRY', 'TYPE']).size().reset_index().rename(columns={0:'Count'}) for year in participants]
type_count_by_country[0]['Year'] = 2013
type_count_by_country[1]['Year'] = 2014
type_count_by_country[2]['Year'] = 2015
type_count_by_country[3]['Year'] = 2016
type_count_by_country[4]['Year'] = 2017
type_count_by_country[5]['Year'] = 2018



type_count_by_country = [year.groupby(['Year', 'COUNTRY', 'TYPE']).sum() for year in type_count_by_country]
type_grouped_by_year_and_country = pd.concat(type_count_by_country).reset_index()

type_grouped_by_year_and_country.to_csv('Type Stats.csv', sep=',', encoding='utf-8', index=False)
type_grouped_by_year_and_country.head()

,Year,COUNTRY,TYPE,Count
0,2013,AR,Nodal,27
1,2013,AR,Partner,1
2,2013,AU,Public,6
3,2013,BE,Regional,1
4,2013,BG,Public,1


---

Add zeros to the type stats:

In [71]:
years = range(2013, 2019)
list(years)

[2013, 2014, 2015, 2016, 2017, 2018]

In [65]:
countries = type_grouped_by_year_and_country['COUNTRY'].unique()
countries

array(['AR', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CL', 'CM', 'CN', 'CO',
       'CR', 'DE', 'DK', 'DO', 'EC', 'ES', 'GB', 'GE', 'GT', 'HU', 'ID',
       'IT', 'JP', 'KE', 'KG', 'KH', 'LA', 'MN', 'MX', 'MY', 'MZ', 'NI',
       'NL', 'NZ', 'PA', 'PE', 'PH', 'PT', 'SE', 'SG', 'US', 'VN', 'BB',
       'BO', 'FR', 'GR', 'IL', 'IN', 'KR', 'LT', 'NO', 'NP', 'TT', 'AE',
       'AT', 'BD', 'BT', 'BW', 'EG', 'ET', 'GH', 'JO', 'KZ', 'LK', 'MA',
       'MM', 'PR', 'RO', 'SI', 'TH', 'TN', 'TZ', 'ZA', 'ZW', 'BJ', 'BS',
       'BZ', 'MQ', 'RU', 'SR', 'SZ', 'MG', 'NG', 'SC', 'CI', 'JM', 'LR',
       'PK'], dtype=object)

In [86]:
types = np.append(type_grouped_by_year_and_country['TYPE'].unique(),('Webinar'))
types

array(['Nodal', 'Partner', 'Public', 'Regional', 'Intro', 'Seminar',
       'Symposium', 'Webinar'], dtype=object)

In [94]:
empty = pd.DataFrame(list(itertools.product(years, countries, types, [0])), columns=['Year', 'Country', 'Type', 'Count'])
display(empty.head(), empty.shape)

,Year,Country,Type,Count
0,2013,AR,Nodal,0
1,2013,AR,Partner,0
2,2013,AR,Public,0
3,2013,AR,Regional,0
4,2013,AR,Intro,0


(4272, 4)

In [103]:
type_grouped_by_year_and_country

,Year,Country,Type,Count
0,2013,AR,Nodal,27
1,2013,AR,Partner,1
2,2013,AU,Public,6
3,2013,BE,Regional,1
4,2013,BG,Public,1
5,2013,BR,Nodal,1
6,2013,BR,Partner,7
7,2013,BR,Public,1
8,2013,CA,Nodal,1
9,2013,CA,Partner,18


In [97]:
type_grouped_by_year_and_country.rename(columns={'COUNTRY': 'Country', 'TYPE': 'Type'}, inplace=True)

In [120]:
type_stats = type_grouped_by_year_and_country.append(empty).groupby(['Year', 'Country', 'Type']).max()
display(type_stats.shape)
type_stats.head(16)

(4272, 1)

Count
Year Country Type            
2013 AE      Intro          0
             Nodal          0
             Partner        0
             Public         0
             Regional       0
             Seminar        0
             Symposium      0
             Webinar        0
     AR      Intro          0
             Nodal         27
             Partner        1
             Public         0
             Regional       0
             Seminar        0
             Symposium      0
             Webinar        0

Add world stats for each year and type:

In [133]:
data = pd.read_excel('../Data/PARTICIPANT LISTS - 2013-present.xlsx', sheet_name=None, header=1)
### People trained

attendance = pd.DataFrame()

# years 
for yr in data.keys():

    attendance = attendance.append({'Year': yr,

                                    # number of participants (P) for year yr
                                    'Total' : len(data[yr][data[yr]['P/I']=='P']), 

                                    # number of participants for year yr per type
                                    'Intro' : len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Intro')]),
                                    'Nodal' : len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Nodal')]),
                                    'Partner':len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Partner')]),
                                    'Public': len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Public')]),
                                    'Regional': len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Regional')]),
                                    'Seminar': len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Seminar')]),
                                    'Symposium': len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Symposium')]),
                                    'Webinar': len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='Webinar')]),
                                    #'Other training': len(data[yr][(data[yr]['P/I']=='P') & (data[yr]['TYPE']=='nan')])
                                   },
                                   ignore_index=True)

attendance['Other trainings'] = attendance['Total'] - attendance['Symposium']   
attendance = attendance.sort_values('Year')
attendance = attendance.set_index('Year')

# Get totals
# attendance = attendance.append(pd.Series(attendance.sum(numeric_only=True), name='Total'), ignore_index=False)

attendance

,Intro,Nodal,Partner,Public,Regional,Seminar,Symposium,Total,Webinar,Other trainings
Year,,,,,,,,,,
2013,131.0,71.0,295.0,38.0,77.0,0.0,120.0,757.0,25.0,637.0
2014,114.0,32.0,50.0,0.0,87.0,0.0,120.0,480.0,77.0,360.0
2015,22.0,68.0,211.0,0.0,0.0,14.0,208.0,541.0,18.0,333.0
2016,0.0,0.0,71.0,0.0,128.0,0.0,228.0,427.0,0.0,199.0
2017,0.0,0.0,437.0,0.0,39.0,0.0,264.0,740.0,0.0,476.0
2018,0.0,9.0,430.0,0.0,16.0,0.0,283.0,739.0,0.0,456.0


In [176]:
att = attendance.drop(columns=['Total', 'Other trainings']).reset_index()
att['Country'] = 'WORLD'
world_stats = att.melt(id_vars=['Year', 'Country'], var_name='Type', value_name='Count')
world_stats['Year'] = pd.to_numeric(world_stats['Year'])
world_stats

,Year,Country,Type,Count
0,2013,WORLD,Intro,131.0
1,2014,WORLD,Intro,114.0
2,2015,WORLD,Intro,22.0
3,2016,WORLD,Intro,0.0
4,2017,WORLD,Intro,0.0
5,2018,WORLD,Intro,0.0
6,2013,WORLD,Nodal,71.0
7,2014,WORLD,Nodal,32.0
8,2015,WORLD,Nodal,68.0
9,2016,WORLD,Nodal,0.0


In [178]:
type_stats = type_stats.reset_index().append(world_stats).groupby(['Year', 'Country', 'Type']).max()
type_stats

Count
Year Country Type            
2013 AE      Intro        0.0
             Nodal        0.0
             Partner      0.0
             Public       0.0
             Regional     0.0
             Seminar      0.0
             Symposium    0.0
             Webinar      0.0
     AR      Intro        0.0
             Nodal       27.0
             Partner      1.0
             Public       0.0
             Regional     0.0
             Seminar      0.0
             Symposium    0.0
             Webinar      0.0
     AT      Intro        0.0
             Nodal        0.0
             Partner      0.0
             Public       0.0
             Regional     0.0
             Seminar      0.0
             Symposium    0.0
             Webinar      0.0
     AU      Intro        0.0
             Nodal        0.0
             Partner      0.0
             Public       6.0
             Regional     0.0
             Seminar      0.0
...                       ...
2018 VN      Partner      0.0
             Public       0.0
             Regional     0.0
             Seminar      0.0
             Symposium    0.0
             Webinar      0.0
     WORLD   Intro        0.0
             Nodal        9.0
             Partner    430.0
             Public       0.0
             Regional    16.0
             Seminar      0.0
             Symposium  283.0
             Webinar      0.0
     ZA      Intro        0.0
             Nodal        0.0
             Partner      0.0
             Public       0.0
             Regional     0.0
             Seminar      0.0
             Symposium    1.0
             Webinar      0.0
     ZW      Intro        0.0
             Nodal        0.0
             Partner      0.0
             Public       0.0
             Regional     0.0
             Seminar      0.0
             Symposium    0.0
             Webinar      0.0

[4320 rows x 1 columns]

Add totals for each country and the world:

In [191]:
totals = type_stats.groupby(['Country', 'Type']).sum().reset_index()
totals['Year'] = 'Total'
cols = totals.columns.tolist()
cols = cols[-1:] + cols[:-1]
totals = totals[cols]
totals

,Year,Country,Type,Count
0,Total,AE,Intro,0.0
1,Total,AE,Nodal,1.0
2,Total,AE,Partner,0.0
3,Total,AE,Public,0.0
4,Total,AE,Regional,0.0
5,Total,AE,Seminar,0.0
6,Total,AE,Symposium,0.0
7,Total,AE,Webinar,0.0
8,Total,AR,Intro,0.0
9,Total,AR,Nodal,27.0


In [195]:
type_stats = type_stats.reset_index().append(totals).groupby(['Year', 'Country', 'Type']).max()
type_stats

Count
Year  Country Type             
2013  AE      Intro         0.0
              Nodal         0.0
              Partner       0.0
              Public        0.0
              Regional      0.0
              Seminar       0.0
              Symposium     0.0
              Webinar       0.0
      AR      Intro         0.0
              Nodal        27.0
              Partner       1.0
              Public        0.0
              Regional      0.0
              Seminar       0.0
              Symposium     0.0
              Webinar       0.0
      AT      Intro         0.0
              Nodal         0.0
              Partner       0.0
              Public        0.0
              Regional      0.0
              Seminar       0.0
              Symposium     0.0
              Webinar       0.0
      AU      Intro         0.0
              Nodal         0.0
              Partner       0.0
              Public        6.0
              Regional      0.0
              Seminar       0.0
...                         ...
Total VN      Partner       8.0
              Public        0.0
              Regional      2.0
              Seminar       0.0
              Symposium     0.0
              Webinar       0.0
      WORLD   Intro       267.0
              Nodal       180.0
              Partner    1494.0
              Public       38.0
              Regional    347.0
              Seminar      14.0
              Symposium  1223.0
              Webinar     120.0
      ZA      Intro         0.0
              Nodal         0.0
              Partner      18.0
              Public        0.0
              Regional      2.0
              Seminar       0.0
              Symposium     3.0
              Webinar       0.0
      ZW      Intro         0.0
              Nodal         0.0
              Partner       1.0
              Public        0.0
              Regional      0.0
              Seminar       0.0
              Symposium     0.0
              Webinar       0.0

[5040 rows x 1 columns]

Sanity check:

In [197]:
len(types) * (len(years) + 1) * (len(countries) + 1) == len(type_stats)

True

Save:

In [198]:
type_stats.to_csv('../Data/Type_Stats.csv')

---

In [49]:
training_location = [year.groupby('COUNTRY.1').size() for year in participants]

aggregated_training_location = pd.concat(training_location, axis=1, sort=True).reset_index().fillna(value=0)
column_mapping = {'index': 'Country', 0: '2013', 1: '2014', 2: '2015', 3: '2016', 4: '2017', 5: '2018'}
aggregated_training_location = aggregated_training_location.rename(index=str, columns=column_mapping)
aggregated_training_location = pd.melt(aggregated_training_location, id_vars=['Country'], value_vars=["2013", "2014", "2015", "2016", "2017", "2018"])
aggregated_training_location = aggregated_training_location.sort_values(by='Country').reset_index(drop=True)
aggregated_training_location = aggregated_training_location.rename(columns={'variable': 'Year', 'value':"Number of Participants"})

aggregated_training_location.to_csv('Training Location Data.csv', sep=',', encoding='utf-8', index=False)
aggregated_training_location.head(10)

,Country,Year,Number of Participants
0,AR,2013,28.0
1,AR,2015,0.0
2,AR,2014,0.0
3,AR,2018,0.0
4,AR,2017,0.0
5,AR,2016,0.0
6,BB,2015,19.0
7,BB,2014,0.0
8,BB,2018,0.0
9,BB,2017,0.0


In [25]:
aggregated_training_location.groupby('Country').sum()

,Number of Participants
Country,
AR,28.0
BB,19.0
BR,20.0
BT,23.0
CA,18.0
CH,42.0
CL,24.0
CN,692.0
CO,18.0
